# Data Preparation  

In [44]:
!poetry add -qqq python-dotenv datasets wandb didkit


In [34]:
import os
from dotenv import load_dotenv, find_dotenv
if os.path.exists('../env'):
    load_dotenv(find_dotenv())
import wandb

In [36]:
wandb.login(key=os.getenv('WANDB_API_KEY'))
run = wandb.init(project="verida-pii", name="verida_data_raw")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb: Network error resolved after 1:13:41.968146, resuming normal operation.
wandb: Network error resolved after 0:42:49.841123, resuming normal operation.
wandb: Network error resolved after 0:18:54.049113, resuming normal operation.


In [62]:
from datasets import load_dataset
import pandas as pd
data_name="Ezi/medical_and_legislators_synthetic"
data = load_dataset(path=data_name, split='train')
data_df = data.to_pandas()
data_df.head()
print(len(data_df))



539


In [30]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
device

'mps'

In [63]:
# DiD Generator
import didkit

def generate_did():
    key = didkit.generate_ed25519_key()
    did = didkit.key_to_did("key", key)
    return did, key

In [64]:
from tqdm import tqdm, tqdm_notebook, tqdm_pandas
import pandas as pd

In [70]:
#data_df['did'] = data_df.apply(lambda x: generate_did()[0], axis=1)
#data_df['key'] = data_df.apply(lambda x: generate_did()[1], axis=1)
cleaned_df = data_df.copy()
cleaned_df.head()
cleaned_df.isna().sum()
cleaned_df.isna().dropna()
cleaned_df.describe()
cleaned_df.shape
cleaned_df.columns

Index(['last_name', 'first_name', 'middle_name', 'suffix', 'nickname',
       'full_name', 'birthday', 'gender', 'type', 'state', 'district',
       'senate_class', 'party', 'url', 'address', 'phone', 'contact_form',
       'rss_url', 'twitter', 'facebook', 'youtube', 'youtube_id',
       'bioguide_id', 'thomas_id', 'opensecrets_id', 'lis_id', 'fec_ids',
       'cspan_id', 'govtrack_id', 'votesmart_id', 'ballotpedia_id',
       'washington_post_id', 'icpsr_id', 'wikipedia_id', 'last_name.1',
       'first_name.1', 'middle_name.1', 'suffix.1', 'nickname.1',
       'full_name.1', 'birthday.1', 'gender.1', 'type.1', 'state.1',
       'district.1', 'senate_class.1', 'party.1', 'url.1', 'address.1',
       'phone.1', 'contact_form.1', 'rss_url.1', 'twitter.1', 'facebook.1',
       'youtube.1', 'youtube_id.1', 'bioguide_id.1', 'thomas_id.1',
       'opensecrets_id.1', 'lis_id.1', 'fec_ids.1', 'cspan_id.1',
       'govtrack_id.1', 'votesmart_id.1', 'ballotpedia_id.1',
       'washington_post_

In [71]:
data_did = data_df.copy()
data_did.to_csv("data_did.csv")
data_did

In [76]:
# Back to dataset

secure_mode = False
train_split = 0.8
test_every = 5
batch_size = 800

ds = data_did
train_len = int(train_split * len(ds))
test_len = len(ds) - train_len

print(f"{train_len} samples for training, {test_len} for testing")

train_ds, test_ds = torch.utils.data.random_split(ds, [train_len, test_len])


ModuleNotFoundError: No module named 'DatasetDict'

In [78]:
from datasets import load_dataset
ds = load_dataset(train_ds, test_ds, split=["train", "test"])


TypeError: expected str, bytes or os.PathLike object, not Subset

In [79]:
import torch
import torch.nn as nn

class CharByteEncoder(nn.Module):
    """
    This encoder takes a UTF-8 string and encodes its bytes into a Tensor. It can also
    perform the opposite operation to check a result.
    Examples:
    >>> encoder = CharByteEncoder()
    >>> t = encoder('Ślusàrski')  # returns tensor([256, 197, 154, 108, 117, 115, 195, 160, 114, 115, 107, 105, 257])
    >>> encoder.decode(t)  # returns "<s>Ślusàrski</s>"
    """

    def __init__(self):
        super().__init__()
        self.start_token = "<s>"
        self.end_token = "</s>"
        self.pad_token = "<pad>"

        self.start_idx = 256
        self.end_idx = 257
        self.pad_idx = 258

    def forward(self, s: str, pad_to=0) -> torch.LongTensor:
        """
        Encodes a string. It will append a start token <s> (id=self.start_idx) and an end token </s>
        (id=self.end_idx).
        Args:
            s: The string to encode.
            pad_to: If not zero, pad by appending self.pad_idx until string is of length `pad_to`.
                Defaults to 0.
        Returns:
            The encoded LongTensor of indices.
        """
        encoded = s.encode()
        n_pad = pad_to - len(encoded) if pad_to > len(encoded) else 0
        return torch.LongTensor(
            [self.start_idx]
            + [c for c in encoded]  # noqa
            + [self.end_idx]
            + [self.pad_idx for _ in range(n_pad)]
        )

    def decode(self, char_ids_tensor: torch.LongTensor) -> str:
        """
        The inverse of `forward`. Keeps the start, end, and pad indices.
        """
        char_ids = char_ids_tensor.cpu().detach().tolist()

        out = []
        buf = []
        for c in char_ids:
            if c < 256:
                buf.append(c)
            else:
                if buf:
                    out.append(bytes(buf).decode())
                    buf = []
                if c == self.start_idx:
                    out.append(self.start_token)
                elif c == self.end_idx:
                    out.append(self.end_token)
                elif c == self.pad_idx:
                    out.append(self.pad_token)

        if buf:  # in case some are left
            out.append(bytes(buf).decode())
        return "".join(out)

    def __len__(self):
        """
        The length of our encoder space. This is fixed to 256 (one byte) + 3 special chars
        (start, end, pad).
        Returns:
            259
        """
        return 259

In [80]:
from torch.nn.utils.rnn import pad_sequence

def padded_collate(batch, padding_idx=0):
    x = pad_sequence(
        [elem[0] for elem in batch], batch_first=True, padding_value=padding_idx
    )
    y = torch.stack([elem[1] for elem in batch]).long()

    return x, y

In [74]:
from torch.utils.data import Dataset
from pathlib import Path


class NamesDataset(Dataset):
    def __init__(self, root):
        self.root = Path(root)

        self.labels = list({langfile.stem for langfile in self.root.iterdir()})
        self.labels_dict = {label: i for i, label in enumerate(self.labels)}
        self.encoder = CharByteEncoder()
        self.samples = self.construct_samples()

    def __getitem__(self, i):
        return self.samples[i]

    def __len__(self):
        return len(self.samples)

    def construct_samples(self):
        samples = []
        for langfile in self.root.iterdir():
            label_name = langfile.stem
            label_id = self.labels_dict[label_name]
            with open(langfile, "r") as fin:
                for row in fin:
                    samples.append(
                        (self.encoder(row.strip()), torch.tensor(label_id).long())
                    )
        return samples

    def label_count(self):
        cnt = Counter()
        for _x, y in self.samples:
            label = self.labels[int(y)]
            cnt[label] += 1
        return cnt


VOCAB_SIZE = 256 + 3  # 256 alternatives in one byte, plus 3 special characters.

In [81]:
# Data Loaders
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    pin_memory=True,
    collate_fn=padded_collate,
)

test_loader = DataLoader(
    test_ds,
    batch_size=2 * batch_size,
    shuffle=False,
    pin_memory=True,
    collate_fn=padded_collate,
)

In [85]:
import pandas as pd

df = pd.read_csv("hf://datasets/synavate/medical_records_did/data_did.csv")

In [93]:
df_drop=df.copy()
df_drop.isnull().drop(index=1)
df_drop.isna().sum()
df_drop.drop(columns=['Unnamed: 0', 'middle_name', 'suffix', 'nickname'], inplace=True)
df_drop.columns

Index(['last_name', 'first_name', 'full_name', 'birthday', 'gender', 'type',
       'state', 'district', 'senate_class', 'party', 'url', 'address', 'phone',
       'contact_form', 'rss_url', 'twitter', 'facebook', 'youtube',
       'youtube_id', 'bioguide_id', 'thomas_id', 'opensecrets_id', 'lis_id',
       'fec_ids', 'cspan_id', 'govtrack_id', 'votesmart_id', 'ballotpedia_id',
       'washington_post_id', 'icpsr_id', 'wikipedia_id', 'last_name.1',
       'first_name.1', 'middle_name.1', 'suffix.1', 'nickname.1',
       'full_name.1', 'birthday.1', 'gender.1', 'type.1', 'state.1',
       'district.1', 'senate_class.1', 'party.1', 'url.1', 'address.1',
       'phone.1', 'contact_form.1', 'rss_url.1', 'twitter.1', 'facebook.1',
       'youtube.1', 'youtube_id.1', 'bioguide_id.1', 'thomas_id.1',
       'opensecrets_id.1', 'lis_id.1', 'fec_ids.1', 'cspan_id.1',
       'govtrack_id.1', 'votesmart_id.1', 'ballotpedia_id.1',
       'washington_post_id.1', 'icpsr_id.1', 'wikipedia_id.1'],
    

In [ ]:
!pip install scikit-learn
from sklearn import train_test_split

